# 如何将运行时配置附加到一个 Runnable 上

:::info 预备知识

本指南假定您已经熟悉以下概念：

- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)
- [链式调用可运行对象（Runnable）](/docs/how_to/sequence/)
- [工具调用](/docs/how_to/tool_calling/)

:::

有时我们想要使用预定义的配置来调用一个 [`Runnable`](https://api.js.langchain.com/classes/langchain_core.runnables.Runnable.html)，而无需调用方指定这些配置。我们可以使用 [`Runnable.withConfig()`](https://api.js.langchain.com/classes/langchain_core.runnables.Runnable.html#withConfig) 方法提前设置这些参数。

## 绑定停止序列

假设我们有一个简单的提示词 + 模型链：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/openai @langchain/core
</Npm2Yarn>
```

In [1]:
import { StringOutputParser } from "@langchain/core/output_parsers";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";

const prompt = ChatPromptTemplate.fromMessages(
    [
        [
            "system",
            "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ],
        ["human", "{equation_statement}"],
    ]
)

const model = new ChatOpenAI({ model: "gpt-4o", temperature: 0 });

const runnable = prompt.pipe(model).pipe(new StringOutputParser());

const res = await runnable.invoke({
  equation_statement: "x raised to the third plus seven equals 12"
});

console.log(res);

EQUATION: x^3 + 7 = 12

SOLUTION: 
Subtract 7 from both sides:
x^3 = 5

Take the cube root of both sides:
x = ∛5


在某些提示技术中，设置一个或多个`stop`（停止）词可能会很有用，当模型生成这些词时将停止生成。在直接使用模型时，我们通过传递给`invoke`的额外`options`参数来设置`stop`词。例如，如果我们想将前面的例子修改为方程格式化工具该怎么办？我们可以指示生成过程在遇到单词`SOLUTION`时停止，这样输出的结果就只会是格式化后的方程。

在这种情况下，我们将模型作为[RunnableSequence](https://api.js.langchain.com/classes/langchain_core.runnables.RunnableSequence.html)的一部分使用。我们无需依赖前面的步骤输出包含停止词的配置，而可以在创建`RunnableSequence`时使用`withConfig`直接绑定所需的配置：

In [2]:
// stop generating after the equation is written
const equationFormatter = prompt
  .pipe(model.withConfig({ stop: ["SOLUTION"] }))
  .pipe(new StringOutputParser());

// generate only the equation, without needing to set the stop word
const formattedEquation = await equationFormatter.invoke({
  equation_statement: "x raised to the third plus seven equals 12"
});

console.log(formattedEquation);

EQUATION: x^3 + 7 = 12




这使得生成的 `Runnable` 管道更易于使用。调用者无需了解所使用的具体提示格式，也无需在调用时指定停止词。他们只需运行自己的查询，就能获得预期的结果。

## 附加 OpenAI 工具

另一个常见的用例是工具调用。虽然对于支持工具调用的模型，你通常应该使用 [`bindTools()`](/docs/how_to/tool_calling/) 方法，但如果你需要更底层的控制，也可以直接绑定特定于提供者的参数：

In [3]:
const tools = [
  {
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA",
          },
          "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
      },
    },
  }
];

const modelWithTools = new ChatOpenAI({ model: "gpt-4o" }).withConfig({ tools });

await modelWithTools.invoke("What's the weather in SF, NYC and LA?")

AIMessage {
  "id": "chatcmpl-BXjkosti03tvSmaxAuYtpRvbEkhRx",
  "content": "",
  "additional_kwargs": {
    "tool_calls": [
      {
        "id": "call_a15PVBt9g3eCHULn7DBRbL9a",
        "type": "function",
        "function": "[Object]"
      },
      {
        "id": "call_bQrHLyJ6fAaNkEPNBtgYfIFb",
        "type": "function",
        "function": "[Object]"
      },
      {
        "id": "call_FxRswXKWou53G0LNQQCvPod4",
        "type": "function",
        "function": "[Object]"
      }
    ]
  },
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 82,
      "completionTokens": 71,
      "totalTokens": 153
    },
    "finish_reason": "tool_calls",
    "model_name": "gpt-4o-2024-08-06",
    "usage": {
      "prompt_tokens": 82,
      "completion_tokens": 71,
      "total_tokens": 153,
      "prompt_tokens_details": {
        "cached_tokens": 0,
        "audio_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0,
        "audio_tokens": 0,

## 下一步

现在您已经了解了如何将运行时参数绑定到一个 Runnable。

接下来，您可能对我们的如何通过链传递数据的指南感兴趣：[通过链传递数据](/docs/how_to/passthrough/)。